In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dmsum\\OneDrive\\Desktop\\End-to-End-Deep-Learning-Project-Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\dmsum\\OneDrive\\Desktop\\End-to-End-Deep-Learning-Project-Chicken-Disease-Classification'

## Update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

## Update the configuration manager in src config

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

## Update the components

In [8]:
from urllib.parse import urlparse
import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [15]:
from pathlib import Path
import tensorflow as tf
import json

def save_json(path: Path, data: dict):
    with open(path, "w") as f:
        json.dump(data, f, indent=4)

class Evaluation:
    def __init__(self, config):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        training_data_path = Path("artifacts/data_ingestion/unzip")

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=training_data_path,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)  # use self.model

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


## Update the pipeline

In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2025-10-09 22:25:25,166: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-09 22:25:25,172: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-09 22:25:25,174: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 56s 7s/step - loss: nan - accuracy: 0.5000
